In [1]:
import dspy
from dspy.teleprompt import *
from dspy.evaluate import Evaluate
import re
import numpy as np
from sklearn.model_selection import train_test_split

os.environ['TOGETHER_API_KEY'] = '35ba5bebf6288e43fdc8989965161592e3335d7067c772c0c6995cdc0e60cd88'
os.environ['TOGETHER_API_BASE'] = 'https://api.together.xyz/v1'

In [2]:
# # Initialize the OpenAI client with your API key
# gpt3_turbo = dspy.OpenAI(
#     model='gpt-3.5-turbo-instruct',
#     api_key='sk-0Mn2APXfVvfQM-nsbyefl6zSxQFyvDfL1rafLF3TqMT3BlbkFJqv-erVjFqV8KIcce6GFXfdF_J3W9fHedyLpMBNEmcA'
# )

# # Configure the default LM (language model) to use
# dspy.configure(lm=gpt3_turbo)

In [3]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3-70B-Instruct-Turbo")
# llama = dspy.Together(model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo")
dspy.configure(lm=llama)

In [4]:
# graphs is a list of strings where each string is the python code
# graph_descriptions is a list of strings where each string is the description for its corresponding graph at the same index in 'graphs'
# data_descriptions is a list of strings where each string is the description of the data

data_descriptions = [
    "Fields: sepal_length (float), sepal_width (float), petal_length (float), petal_width (float), species (string). Format: CSV",
    "Fields: survived (integer), pclass (integer), name (string), sex (string), age (float), sibsp (integer), parch (integer), ticket (string), fare (float), cabin (string), embarked (string). Format: CSV",
    "Fields: Month (string), Temperature (float), Rainfall (float), Humidity (float). Format: CSV",
    "Fields: species (string), island (string), bill_length_mm (float), bill_depth_mm (float), flipper_length_mm (float), body_mass_g (float), sex (string). Format: CSV",
    "Fields: fixed_acidity (float), volatile_acidity (float), citric_acid (float), residual_sugar (float), chlorides (float), free_sulfur_dioxide (float), total_sulfur_dioxide (float), density (float), pH (float), sulphates (float), alcohol (float), quality (integer). Format: CSV",
    "Fields: year (integer), month (string), passengers (integer). Format: CSV",
    "Fields: Product (string), Q1 Sales (integer), Q2 Sales (integer), Q3 Sales (integer), Q4 Sales (integer), Price (float). Format: CSV",
    "Fields: age (float), sex (integer), cp (integer), trestbps (integer), chol (integer), fbs (integer), restecg (integer), thalach (integer), exang (integer), oldpeak (float), slope (integer), ca (integer), thal (integer), target (integer). Format: CSV",
    "Fields: tconst (string), averageRating (float), numVotes (integer), titleType (string), primaryTitle (string), originalTitle (string), isAdult (string), startYear (integer), endYear (string), runtimeMinutes (string), genres (string). Format: TSV",
    "Fields: Entity (string), Year (integer), Suicide rates (float), Crude suicide rates per gender (float), Country (string), Age standardized (boolean). Format: CSV. Coverage: 198 countries, 1950-2011",
    "Fields: Country (string), Year (integer), Sex (string), Suicide rates (float). Format: CSV. Coverage: 2013-2015",
    "Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.",
    "Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.",
    "Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.",
    "Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.",
    "Fields: Country (string), Year (integer), Status (string), Life expectancy (float), Adult Mortality (float), Infant deaths (integer), Alcohol (float), Percentage expenditure (float), Hepatitis B (float), Measles (integer), BMI (float), Under-five deaths (integer), Polio (float), Total expenditure (float), Diphtheria (float), HIV/AIDS (float), GDP (float), Population (float), Thinness 1-19 years (float), Thinness 5-9 years (float), Income composition of resources (float), Schooling (float). Format: CSV",
    "Fields: Country (string), Year (integer), Status (string), Life expectancy (float), Adult Mortality (float), Infant deaths (integer), Alcohol (float), Percentage expenditure (float), Hepatitis B (float), Measles (integer), BMI (float), Under-five deaths (integer), Polio (float), Total expenditure (float), Diphtheria (float), HIV/AIDS (float), GDP (float), Population (float), Thinness 1-19 years (float), Thinness 5-9 years (float), Income composition of resources (float), Schooling (float). Format: CSV",
    "Fields: Country (string), Year (integer), Status (string), Life expectancy (float), Adult Mortality (float), Infant deaths (integer), Alcohol (float), Percentage expenditure (float), Hepatitis B (float), Measles (integer), BMI (float), Under-five deaths (integer), Polio (float), Total expenditure (float), Diphtheria (float), HIV/AIDS (float), GDP (float), Population (float), Thinness 1-19 years (float), Thinness 5-9 years (float), Income composition of resources (float), Schooling (float). Format: CSV",
    "Fields: age (integer), sex (integer), cp (integer), trtbps (integer), chol (integer), fbs (integer), restecg (integer), thalachh (integer), exng (integer), oldpeak (float), slp (integer), caa (integer), thall (integer), output (integer). Format: CSV",
    "Fields: age (integer), sex (integer), cp (integer), trtbps (integer), chol (integer), fbs (integer), restecg (integer), thalachh (integer), exng (integer), oldpeak (float), slp (integer), caa (integer), thall (integer), output (integer). Format: CSV",
    "Fields: age (integer), sex (integer), cp (integer), trtbps (integer), chol (integer), fbs (integer), restecg (integer), thalachh (integer), exng (integer), oldpeak (float), slp (integer), caa (integer), thall (integer), output (integer). Format: CSV",
    "Fields: age (integer), sex (integer), cp (integer), trtbps (integer), chol (integer), fbs (integer), restecg (integer), thalachh (integer), exng (integer), oldpeak (float), slp (integer), caa (integer), thall (integer), output (integer). Format: CSV",
    "Fields: sl_no (integer), gender (string), ssc_p (float), ssc_b (string), hsc_p (float), hsc_b (string), hsc_s (string), degree_p (float), degree_t (string), workex (string), etest_p (float), specialisation (string), mba_p (float), status (string), salary (float), avg_p (float). Format: CSV",
    "Fields: sl_no (integer), gender (string), ssc_p (float), ssc_b (string), hsc_p (float), hsc_b (string), hsc_s (string), degree_p (float), degree_t (string), workex (string), etest_p (float), specialisation (string), mba_p (float), status (string), salary (float), avg_p (float). Format: CSV",
    "Fields: sl_no (integer), gender (string), ssc_p (float), ssc_b (string), hsc_p (float), hsc_b (string), hsc_s (string), degree_p (float), degree_t (string), workex (string), etest_p (float), specialisation (string), mba_p (float), status (string), salary (float), avg_p (float). Format: CSV",
    "Fields: sl_no (integer), gender (string), ssc_p (float), ssc_b (string), hsc_p (float), hsc_b (string), hsc_s (string), degree_p (float), degree_t (string), workex (string), etest_p (float), specialisation (string), mba_p (float), status (string), salary (float), avg_p (float). Format: CSV",
    "Fields in rating.csv: userId (integer), movieId (integer), rating (float), timestamp (integer). Format: CSV",
    "Fields in rating.csv: userId (integer), movieId (integer), rating (float), timestamp (integer). Format: CSV. Fields in movie.csv: movieId (integer), title (string), genres (string). Format: CSV",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains information about Netflix titles including movies and TV shows. It includes details such as title, director, cast, country, date added to Netflix, release year, rating, duration, genre(s), and description.",
    "This dataset contains Tesla stock data from 2016 to 2021. It includes columns such as Date, Open, High, Low, Close, Adjusted Close, and Volume.",
    "Fields athlete_events: ID (integer), Name (string), Sex (string), Age (float), Height (float), Weight (float), Team (string), NOC (string), Games (string), Year (integer), Season (string), City (string), Sport (string), Event (string), Medal (string). Format: CSV. Fields noc_regions: NOC (string), region (string), notes (string). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: S.NO (integer), Name (string), Nationality (string), Current Rank (integer), Previous Year Rank (string), Sport (string), Year (integer), earnings ($ million) (float). Format: CSV",
    "Fields: data_dte (string), Year (integer), Month (integer), usg_apt_id (integer), usg_apt (string), usg_wac (integer), fg_apt_id (integer), fg_apt (string), fg_wac (integer), airlineid (integer), carrier (string), carriergroup (integer), type (string), Scheduled (integer), Charter (integer), Total (integer). Format: CSV",
    "Fields: data_dte (string), Year (integer), Month (integer), usg_apt_id (integer), usg_apt (string), usg_wac (integer), fg_apt_id (integer), fg_apt (string), fg_wac (integer), airlineid (integer), carrier (string), carriergroup (integer), type (string), Scheduled (integer), Charter (integer), Total (integer). Format: CSV",
    "Fields: data_dte (string), Year (integer), Month (integer), usg_apt_id (integer), usg_apt (string), usg_wac (integer), fg_apt_id (integer), fg_apt (string), fg_wac (integer), airlineid (integer), carrier (string), carriergroup (integer), type (string), Scheduled (integer), Charter (integer), Total (integer). Format: CSV",
    "Fields: data_dte (string), Year (integer), Month (integer), usg_apt_id (integer), usg_apt (string), usg_wac (integer), fg_apt_id (integer), fg_apt (string), fg_wac (integer), airlineid (integer), carrier (string), carriergroup (integer), type (string), Scheduled (integer), Charter (integer), Total (integer). Format: CSV",
    "Fields: data_dte (string), Year (integer), Month (integer), usg_apt_id (integer), usg_apt (string), usg_wac (integer), fg_apt_id (integer), fg_apt (string), fg_wac (integer), airlineid (integer), carrier (string), carriergroup (integer), type (string), Scheduled (integer), Charter (integer), Total (integer). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
    "Fields: Timestamp (string), Age (float), Gender (string), City (string), Position (string), Total years of experience (float), Years of experience in Germany (float), Seniority level (string), Your main technology / programming language (string), Other technologies/programming languages you use often (string), Yearly brutto salary (without bonus and stocks) in EUR (float), Number of vacation days (float), Employment status (string), Сontract duration (string), Main language at work (string), Company size (string), Company type (string), Have you lost your job due to the coronavirus outbreak? (string). Format: CSV",
]


graph_descriptions = [
    "Scatter plot of sepal length vs sepal width, with a new column for sepal area.",
    "Bar plot of survival rate by passenger class, with imputed missing age values and categorized age groups.",
    "Line plot of monthly temperature, with a secondary y-axis bar chart for rainfall, and scatter plot for average humidity.",
    "Pair plot of penguin measurements, with imputed missing values.",
    "Histogram of wine quality, with a new column for categorized alcohol levels.",
    "Line plot of cumulative number of passengers over time.",
    "Grouped bar chart of quarterly sales for each product, with a line plot overlay for total annual sales.",
    "Box plot of cholesterol levels by chest pain type, with outlier detection and removal.",
    "Bar plot of average movie ratings by year, with timestamp conversion and year extraction.",
    "Bar plot showing the top countries with the highest suicide rates for the years 2000, 2005, 2010, and 2015.",
    "Bar plot showing the top countries with the highest suicide rates for males for the years 2000, 2005, 2010, and 2015.",
    "Bar plot showing the total number of Confirmed, Deaths, Recovered, and Active cases around the world.",
    "Bar and pie charts showing the confirmed cases grouped by WHO Regions.",
    "Bar and pie charts showing the deaths grouped by WHO Regions.",
    "Bar chart showing the recovered and active cases grouped by WHO Regions.",
    "Histogram/bar graph of sampled columns to show distribution of data.",
    "Correlation matrix to show relationships between different numerical features.",
    "Scatter and density plots to visualize relationships and distributions of numerical features.",
    "Pie chart showing the distribution of the target variable (output) representing heart attack diagnosis.",
    "Pie charts showing the distribution of binary variables.",
    "Histograms and bar plots showing the distribution of numerical and categorical features respectively.",
    "Heatmap showing the correlation matrix between the features.",
    "Pie chart showing the distribution of placement status (status).",
    "Pie charts showing the distribution of gender, specialization, and work experience.",
    "Bar plots and histograms showing the distribution of numerical and categorical features.",
    "Heatmap showing the correlation matrix between the features.",
    "Histogram of the distribution of movie ratings.",
    "Boxplot of the top 10 most rated movies.",
    "Histogram showing the distribution of movie ratings.",
    "Countplot showing the distribution of release years of Netflix titles.",
    "Bar plot showing the top 10 directors with the most titles on Netflix.",
    "Countplot showing the distribution of movies and TV shows on Netflix.",
    "Countplot showing the distribution of ratings of Netflix titles.",
    "Bar plot showing the top 10 genres on Netflix.",
    "Plot showing the relationship between the opening and closing values of Tesla stock for the training dataset.",
    "Bar plot of the top 10 participating countries in the Olympics for both Summer and Winter seasons.",
    "Bar plot of the number of athletes from each country.",
    "Scatter plot of the unique number of athletes in each country, color-coded by sport.",
    "Bar plot of total earnings per country.",
    "Line plot of total earnings over time by athletes.",
    "Bar plot of total earnings per sport.",
    "Bar plot of the top 5 highest earning athletes.",
    "Line plot of Floyd Mayweather's earnings over the years.",
    "Bar plot of Floyd Mayweather's earnings over the years.",
    "Line plot of Tiger Woods's earnings over the years.",
    "Bar plot of Tiger Woods's earnings over the years.",
    "Bar plot of total earnings per sport for athletes from the USA.",
    "Bar plot of total earnings per sport for athletes from countries other than the USA.",
    "Bar plot of the earnings of the richest soccer players globally.",
    "Line plot of the relationship between rank and earnings.",
    "Count plot of the distribution of flight types (Departures and Passengers).",
    "Relational plot of total flights by year.",
    "Relational plot of total flights by year with carrier group hue.",
    "Bar plot of total flights by year with type hue.",
    "Line plot of scheduled and chartered flights per year.",
    "KDE plot of the distribution of total years of experience.",
    "Scatter plot of age vs total years of experience, colored by gender.",
    "Box plot of gender vs age at market entry.",
    "Histograms of seniority levels for males and females.",
    "Box plot of seniority level vs age.",
    "Histogram of seniority level vs city.",
    "Box plot of gender vs yearly brutto salary.",
    "Box plot of main technology vs yearly brutto salary.",
    "Scatter plot of years of work abroad vs yearly brutto salary.",
    "Box plot of company size vs yearly brutto salary."
]

scores = [3,2,2,3,2,3,3,3,2,1,1,3,3,3,3,1,1,1,3,2,2,3,1,1,3,3,3,3,2,3,3,3,3,1,2,2,2,3,3,3,3,3,3,3,3,3,3,3,3,3,1,1,1,3,2,1,3,3,3,2,2,3,3,2,2]

# kaggle_scores=[1,1], [3,3,3,3], [1,1,1], [3,2,2,3], [1,1,3,3], [3,3,2,3 (last two not included) ], [2,3,3,3,3,1], [2], [2], [2,3,3,3,3,3,3,3,3,3,3,3,3,3], [1,1,1,3,2], [1,3,3,3,2,2,3,3,2,2]

In [5]:
gold_std_graphs = [
    """
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
iris = sns.load_dataset('iris')

# Data preprocessing
iris['sepal_area'] = iris['sepal_length'] * iris['sepal_width']

# Plotting
sns.scatterplot(data=iris, x='sepal_length', y='sepal_width', hue='species', size='sepal_area', sizes=(20, 200))
plt.title('Sepal Length vs Sepal Width by Species')
plt.show()
    """,
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
titanic = sns.load_dataset('titanic')

# Data preprocessing
titanic['age'] = titanic['age'].fillna(titanic['age'].median())
titanic['AgeGroup'] = pd.cut(titanic['age'], bins=[0, 12, 18, 35, 60, 80], labels=['Child', 'Teenager', 'Adult', 'Middle-aged', 'Senior'])

# Plotting
sns.barplot(data=titanic, x='pclass', y='survived', hue='AgeGroup')
plt.title('Survival Rate by Passenger Class and Age Group')
plt.show()
    """,
    """
import matplotlib.pyplot as plt
import pandas as pd

# Save to CSV
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)

# Data preprocessing
data = pd.read_csv('data.csv')
monthly_avg_humidity = data.groupby('Month')['Humidity'].mean().reindex(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Plotting
fig, ax1 = plt.subplots()

ax2 = ax1.twinx()
ax1.plot(data['Month'], data['Temperature'], 'r-', label='Temperature')
ax2.bar(data['Month'], data['Rainfall'], alpha=0.5, label='Rainfall')
ax2.scatter(monthly_avg_humidity.index, monthly_avg_humidity, color='b', label='Avg Humidity')

ax1.set_xlabel('Month')
ax1.set_ylabel('Temperature (C)', color='r')
ax2.set_ylabel('Rainfall (mm) and Avg Humidity (%)', color='b')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')

plt.title('Monthly Temperature, Rainfall, and Humidity')
plt.show()
    """,
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
penguins = sns.load_dataset('penguins')

# Data preprocessing
penguins.fillna(penguins.mean(numeric_only=True), inplace=True)

# Plotting
sns.pairplot(data=penguins, hue='species')
plt.suptitle('Pair Plot of Penguin Measurements with Imputed Missing Values', y=1)
plt.show()
    """,
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
red_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv', sep=';')
white_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv', sep=';')

# Data preprocessing
red_wine['alcohol_level'] = pd.cut(red_wine['alcohol'], bins=[0, 10, 12, 14], labels=['Low', 'Medium', 'High'])
white_wine['alcohol_level'] = pd.cut(white_wine['alcohol'], bins=[0, 10, 12, 14], labels=['Low', 'Medium', 'High'])

# Plotting
sns.histplot(red_wine['quality'], kde=True, color='red', label='Red Wine')
sns.histplot(white_wine['quality'], kde=True, color='yellow', label='White Wine')
plt.title('Wine Quality Distribution with Alcohol Levels')
plt.legend()
plt.show()
    """,
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the dataset
flights = sns.load_dataset('flights')

# Data preprocessing
flights['cumulative_passengers'] = flights['passengers'].cumsum()

# Plotting
sns.lineplot(data=flights, x='year', y='cumulative_passengers')
plt.title('Cumulative Number of Passengers Over Time')
plt.show()
    """,
    """
import pandas as pd

# Create example data
data = {
    'Product': ['A', 'B', 'C', 'D', 'E'],
    'Q1 Sales': [150, 200, 300, 400, 250],
    'Q2 Sales': [160, 210, 320, 420, 260],
    'Q3 Sales': [170, 220, 340, 440, 270],
    'Q4 Sales': [180, 230, 360, 460, 280],
    'Price': [10, 15, 20, 25, 30]
}

# Save to CSV
df = pd.DataFrame(data)
df.to_csv('data.csv', index=False)

import pandas as pd
import matplotlib.pyplot as plt

# Data preprocessing
data = pd.read_csv('data.csv')
data.set_index('Product', inplace=True)
data['Total Sales'] = data[['Q1 Sales', 'Q2 Sales', 'Q3 Sales', 'Q4 Sales']].sum(axis=1)

# Plotting
ax = data[['Q1 Sales', 'Q2 Sales', 'Q3 Sales', 'Q4 Sales']].plot(kind='bar')
ax2 = ax.twinx()
ax2.plot(data.index, data['Total Sales'], 'r-', marker='o', label='Total Sales')
ax.set_xlabel('Product')
ax.set_ylabel('Quarterly Sales')
ax2.set_ylabel('Total Annual Sales')
plt.title('Quarterly Sales and Total Annual Sales by Product')
ax.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
heart_disease = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data', header=None, na_values="?")
heart_disease.columns = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target']

# Data preprocessing
heart_disease.dropna(inplace=True)  # Remove rows with missing values
Q1 = heart_disease['chol'].quantile(0.25)
Q3 = heart_disease['chol'].quantile(0.75)
IQR = Q3 - Q1
heart_disease = heart_disease[(heart_disease['chol'] >= (Q1 - 1.5 * IQR)) & (heart_disease['chol'] <= (Q3 + 1.5 * IQR))]

# Plotting
sns.boxplot(data=heart_disease, x='cp', y='chol')
plt.xlabel('Chest Pain Type')
plt.ylabel('Cholesterol Level')
plt.title('Cholesterol Levels by Chest Pain Type with Outliers Removed')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the datasets
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', dtype={'tconst': 'str', 'averageRating': 'float64', 'numVotes': 'int64'})
movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', dtype={'tconst': 'str', 'titleType': 'str', 'primaryTitle': 'str', 'originalTitle': 'str', 'isAdult': 'str', 'startYear': 'str', 'endYear': 'str', 'runtimeMinutes': 'str', 'genres': 'str'}, low_memory=False)

# Data preprocessing
movies['startYear'] = pd.to_numeric(movies['startYear'], errors='coerce')
ratings = ratings.merge(movies, on='tconst')
ratings = ratings[ratings['startYear'] > 0]  # Remove movies with invalid years
avg_ratings = ratings.groupby('startYear')['averageRating'].mean()

# Plotting
avg_ratings.plot(kind='bar', figsize=(20, 8), color='skyblue')
plt.xlabel('Year')
plt.ylabel('Average Rating')
plt.title('Average Movie Ratings by Year')
plt.show()
    """,
    """
import plotly.offline as py
import plotly.graph_objs as go

t2000 = go.Bar(
    x = total2000[total2000['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2000[total2000['Country'].isin(total2015['Country'].head(10))]['Total suicide rate'].head(10),
    marker=dict(color='rgb(129, 67, 116)'),
    name = '2000'
)

t2005 = go.Bar(
    x = total2005[total2005['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2005[total2005['Country'].isin(total2015['Country'].head(10))]['Total suicide rate'].head(10),
    marker=dict(color='rgb(81, 163, 157)'),
    name = '2005'
)

t2010 = go.Bar(
    x = total2010[total2010['Country'].isin(total2015['Country'].head(10))]['Country'].head(10),
    y = total2010[total2010['Country'].head(10)]['Total suicide rate'].head(10),
    marker=dict(color='rgb(183, 105, 92)'),
    name = '2010'
)

t2015 = go.Bar(
    x = total2015['Country'].head(10),
    y = total2015['Total suicide rate'].head(10),
    marker=dict(color='rgb(205, 187, 121)'),
    name = '2015'
)

data = [t2000, t2005, t2010, t2015]

layout = go.Layout(
    title='Top countries with the highest suicide rates for both sexes',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')
    """,
    """
import plotly.offline as py
import plotly.graph_objs as go

t2000 = go.Bar(
    x = male2000[male2000['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2000[male2000['Country'].head(10)]['Male suicide rate'].head(10),
    marker=dict(color='rgb(55, 83, 109)'),
    name = '2000'
)

t2005 = go.Bar(
    x = male2005[male2005['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2005[male2005['Country'].head(10)]['Male suicide rate'].head(10),
    marker=dict(color='rgb(133, 133, 173)'),
    name = '2005'
)

t2010 = go.Bar(
    x = male2010[male2010['Country'].isin(male2015['Country'].head(10))]['Country'].head(10),
    y = male2010[male2010['Country'].head(10)]['Male suicide rate'].head(10),
    marker=dict(color='rgb(101, 190, 190)'),
    name = '2010'
)

t2015 = go.Bar(
    x = male2015['Country'].head(10),
    y = male2015['Male suicide rate'].head(10),
    marker=dict(color='rgb(26, 118, 255)'),
    name = '2015'
)

data = [t2000, t2005, t2010, t2015]

layout = go.Layout(
    title='Top countries with the highest suicide rates for males',
    xaxis={'title':'Countries'},
    yaxis={'title':'Suicide rate (per 100,000 people)'},
    barmode='group',
    bargap=0.15,
    bargroupgap=0.1
)

fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='basic-bar')
    """,
     """
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')

# Plotting total number of Confirmed, Deaths, Recovered, and Active cases
status_count = ['Confirmed', 'Deaths', 'Recovered', 'Active']
column_sums = df[status_count].sum().plot(kind='bar', xlabel='Status', ylabel='Count in Millions', title='Count of Population Status')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')

# Grouping by WHO Region
Region = df.groupby(by='WHO Region').sum()

# Bar plot for Confirmed Cases by WHO Region
Region['Confirmed'].plot(kind='bar')
plt.title('Confirmed Cases Grouped by WHO Regions')
plt.ylabel('Count in Millions')
plt.show()

# Pie chart for Confirmed Cases by WHO Region
Region['Confirmed'].plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Cases by WHO Region')
plt.ylabel('')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')

# Grouping by WHO Region
Region = df.groupby(by='WHO Region').sum()

# Bar plot for Deaths by WHO Region
Region['Deaths'].plot(kind='bar')
plt.title('Deaths Grouped by WHO Region')
plt.ylabel('Count')
plt.show()

# Pie chart for Deaths by WHO Region
Region['Deaths'].plot(kind='pie', autopct='%1.1f%%')
plt.title('Distribution of Deaths by Region')
plt.xlabel('Region')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')

# Grouping by WHO Region
Region = df.groupby(by='WHO Region').sum()

# Bar plot for Recovered and Active Cases by WHO Regions
Region[['Recovered', 'Active']].plot(kind='bar')
plt.title('Recovered and Active Cases by Regions')
plt.xlabel('Region')
plt.ylabel('Count in millions')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the dataset
df1 = pd.read_csv('../input/Life Expectancy Data.csv', delimiter=',', nrows=1000)
df1.dataframeName = 'Life Expectancy Data.csv'

# Distribution graphs (histogram/bar graph) of sampled columns
def plotPerColumnDistribution(df, nGraphShown, nGraphPerRow):
    nunique = df.nunique()
    df = df[[col for col in df if nunique[col] > 1 and nunique[col] < 50]]  # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df.shape
    columnNames = list(df)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num=None, figsize=(6 * nGraphPerRow, 8 * nGraphRow), dpi=80, facecolor='w', edgecolor='k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = df.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation=90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad=1.0, w_pad=1.0, h_pad=1.0)
    plt.show()

# Call the function to plot distributions
plotPerColumnDistribution(df1, 10, 5)
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the dataset
df1 = pd.read_csv('../input/Life Expectancy Data.csv', delimiter=',', nrows=1000)
df1.dataframeName = 'Life Expectancy Data.csv'

# Correlation matrix
def plotCorrelationMatrix(df, graphWidth):
    filename = df.dataframeName
    df = df.dropna('columns')  # drop columns with NaN
    df = df[[col for col in df if df[col].nunique() > 1]]  # keep columns where there are more than 1 unique values
    if df.shape[1] < 2:
        print(f'No correlation plots shown: The number of non-NaN or constant columns ({df.shape[1]}) is less than 2')
        return
    corr = df.corr()
    plt.figure(num=None, figsize=(graphWidth, graphWidth), dpi=80, facecolor='w', edgecolor='k')
    corrMat = plt.matshow(corr, fignum=1)
    plt.xticks(range(len(corr.columns)), corr.columns, rotation=90)
    plt.yticks(range(len(corr.columns)), corr.columns)
    plt.gca().xaxis.tick_bottom()
    plt.colorbar(corrMat)
    plt.title(f'Correlation Matrix for {filename}', fontsize=15)
    plt.show()

# Call the function to plot the correlation matrix
plotCorrelationMatrix(df1, 8)
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Load the dataset
df1 = pd.read_csv('../input/Life Expectancy Data.csv', delimiter=',', nrows=1000)
df1.dataframeName = 'Life Expectancy Data.csv'

# Scatter and density plots
def plotScatterMatrix(df, plotSize, textSize):
    df = df.select_dtypes(include=[np.number])  # keep only numerical columns
    # Remove rows and columns that would lead to df being singular
    df = df.dropna('columns')
    df = df[[col for col in df if df[col].nunique() > 1]]  # keep columns where there are more than 1 unique values
    columnNames = list(df)
    if len(columnNames) > 10:  # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df = df[columnNames]
    ax = pd.plotting.scatter_matrix(df, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k=1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()

# Call the function to plot scatter and density plots
plotScatterMatrix(df1, 20, 10)
    """,
    """
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

# Explore the distribution of the target variable
target = 'output'
px.pie(df, names=target, title='Distribution of Heart Attack', labels={target: 'Diagnosis'})
    """,
    """
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

# Automatically detect binary variables
binary_vars = [col for col in df.columns if df[col].nunique() == 2]
for col in binary_vars:
    px.pie(df, names=col, title=f"Distribution of {col}")
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

# Detect categorical variables (those with fewer than 11 unique values or already categorical)
binary_vars = [col for col in df.columns if df[col].nunique() == 2]
categorical_vars = [col for col in df.columns if (df[col].nunique() < 11 and col not in binary_vars)]

# Columns to be scaled (excluding binary and categorical)
cols2scale = [col for col in df.columns if col not in binary_vars + categorical_vars]

# Combine columns to plot: numerical columns for histograms, categorical for bar plots
columns_to_plot = cols2scale + categorical_vars

# Number of subplots
num_cols = len(columns_to_plot)
num_rows = (num_cols + 2) // 2  # This creates a grid with 2 columns

# Create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(10, num_rows * 4.7))
axes = axes.flatten()
colors = sns.color_palette("hls", num_cols)

# Loop through the columns to create histograms or bar plots
for i, col in enumerate(columns_to_plot):
    if col in categorical_vars:
        sns.countplot(x=col, data=df, ax=axes[i], palette=[colors[i]])
    else:
        sns.histplot(df[col], kde=True, ax=axes[i], color=colors[i])
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

# Remove any empty subplots
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv")

# Explore correlations between features
plt.figure(figsize=(10,10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()
    """,
        """
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

# Explore the distribution of placement status
status = 'status'
px.pie(df, names=status, title='Distribution of Placement Status', labels={status: 'Placement Status'})
    """,
    """
import pandas as pd
import plotly.express as px

# Load the dataset
df = pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

# Define categorical variables of interest
categorical_vars = ['gender', 'specialisation', 'workex']

for col in categorical_vars:
    px.pie(df, names=col, title=f"Distribution of {col.capitalize()}")
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

# Define numerical and categorical columns
numerical_vars = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary', 'avg_p']
categorical_vars = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']

# Number of subplots
num_vars = len(numerical_vars) + len(categorical_vars)
num_rows = (num_vars + 1) // 2

# Create the subplots
fig, axes = plt.subplots(num_rows, 2, figsize=(14, num_rows * 4))
axes = axes.flatten()

# Plot numerical variables
for i, col in enumerate(numerical_vars):
    sns.histplot(df[col], kde=True, ax=axes[i], color="skyblue")
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Frequency')

# Plot categorical variables
for i, col in enumerate(categorical_vars, start=len(numerical_vars)):
    sns.countplot(x=col, data=df, ax=axes[i], palette="Set2")
    axes[i].set_title(f'Distribution of {col}')
    axes[i].set_xlabel(col)
    axes[i].set_ylabel('Count')

# Adjust layout
plt.tight_layout()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
df = pd.read_csv("../input/factors-affecting-campus-placement/Placement_Data_Full_Class.csv")

# Explore correlations between features
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Matrix')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load movie ratings data
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')

# Visualization of rating distribution
plt.figure(figsize=(10, 6))
sns.histplot(ratings['rating'], bins=50, kde=False)
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Ratings')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load movie ratings and movie information data
ratings = pd.read_csv('/kaggle/input/movielens-20m-dataset/rating.csv')
movies = pd.read_csv('/kaggle/input/movielens-20m-dataset/movie.csv')

# Visualization of top-10 most rated movies
top_movies = ratings['movieId'].value_counts().head(10).index
top_movies_titles = movies[movies['movieId'].isin(top_movies)]
top_movies_ratings = ratings[ratings['movieId'].isin(top_movies)]

plt.figure(figsize=(12, 8))
sns.boxplot(x='movieId', y='rating', data=top_movies_ratings)
plt.xticks(ticks=np.arange(10), labels=top_movies_titles['title'], rotation=90)
plt.xlabel('Movie Title')
plt.ylabel('Rating')
plt.title('Boxplot of Top 10 Most Rated Movies')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
file_path = '/kaggle/input/netflix-shows/netflix_titles.csv'
data = pd.read_csv(file_path)

# Check for missing values
missing_values = data.isnull().sum()
print("\nMissing values in each column:")
display(missing_values)

# Data types of each column
print("\nData types of each column:")
display(data.dtypes)

# Unique values in each column
print("\nUnique values in each column:")
for column in data.columns:
    print(f"{column}: {data[column].nunique()} unique values")

# Distribution of categorical features
categorical_features = data.select_dtypes(include=[object]).columns.tolist()
print("\nDistribution of categorical features:")
for column in categorical_features:
    if data[column].nunique() <= 20:
        plt.figure(figsize=(10, 5))
        sns.countplot(y=data[column], order=data[column].value_counts().index)
        plt.title(f'Distribution of {column}')
        plt.show()
    else:
        print(f"{column} has more than 20 unique values, skipping plot.")
    """,
    """
# Distribution of release years
plt.figure(figsize=(12, 6))
sns.countplot(y='release_year', data=data, order=data['release_year'].value_counts().index)
plt.title('Distribution of Release Years')
plt.xlabel('Count')
plt.ylabel('Release Year')
plt.show()
    """,
    """
# Top 10 directors
top_directors = data['director'].value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_directors.values, y=top_directors.index, palette='viridis')
plt.title('Top 10 Directors on Netflix')
plt.xlabel('Number of Movies/TV Shows')
plt.ylabel('Director')
plt.show()
    """,
    """
# Movie vs TV Show
plt.figure(figsize=(10, 5))
sns.countplot(x='type', data=data)
plt.title('Distribution of Movies and TV Shows')
plt.xlabel('Type')
plt.ylabel('Count')
plt.show()
    """,
    """
# Movie ratings
plt.figure(figsize=(12, 6))
sns.countplot(y='rating', data=data, order=data['rating'].value_counts().index)
plt.title('Distribution of Ratings')
plt.xlabel('Count')
plt.ylabel('Rating')
plt.show()
    """,
    """
# Top 10 genres
data['listed_in'] = data['listed_in'].str.split(', ')
top_genres = data['listed_in'].explode().value_counts().head(10)
plt.figure(figsize=(12, 6))
sns.barplot(x=top_genres.values, y=top_genres.index, palette='coolwarm')
plt.title('Top 10 Genres on Netflix')
plt.xlabel('Number of Movies/TV Shows')
plt.ylabel('Genre')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv("../input/tesla-stock-data-20162021/TSLA.csv")

# Display the dataset
print(data)

# Select features and target
X = data[['Open', 'High', 'Low', 'Volume']]
y = data['Close']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=34)

# Create linear regression model
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
r_sq = regr.score(X_test, y_test)

print('Training Score:')
r_sq1 = regr.score(X_train, y_train)
print(r_sq1)

print('Testing Score:')
print(r_sq)

# Predict the closing values
predicted = regr.predict(X_test)

print('Predicted closing values are:')
print(predicted)

# Plot the relationship between opening and closing values for the training dataset
plt.plot(list(X_train["Open"]), y_train)
plt.xlabel('Opening values')
plt.ylabel('Closing values')
plt.title('Analysis of Closing with respect to opening for the training dataset')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Load the data
df1 = pd.read_csv("../input/120-years-of-olympic-history-athletes-and-results/athlete_events.csv")
df2 = pd.read_csv("../input/120-years-of-olympic-history-athletes-and-results/noc_regions.csv")

# Merging the two tables
df = df1.merge(df2, how="left", on='NOC')

# Top 10 participating countries in Summer Olympics
summer = df[df['Season'] == 'Summer'][['Team', 'Season']]
Top_10_summer = summer['Team'].value_counts().sort_values(ascending=False).head(10)

plt.figure(figsize=(20, 10))
plt.title("Top 10 participating countries in Summer Olympics")
plt.xlabel("Countries")
plt.ylabel("Number of participants")
sns.barplot(x=Top_10_summer.index, y=Top_10_summer, palette='Set2')
plt.show()

# Top 10 participating countries in Winter Olympics
winter = df[df['Season'] == 'Winter'][['Team', 'Season']]
Top_10_winter = winter['Team'].value_counts().sort_values(ascending=False).head(10)

plt.figure(figsize=(20, 10))
plt.title("Top 10 participating countries in Winter Olympics")
plt.xlabel("Countries")
plt.ylabel("Number of participants")
sns.barplot(x=Top_10_winter.index, y=Top_10_winter, palette='Set2')
plt.show()
    """,
"""
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of the number of athletes from each country
df.groupby("Nationality")["S.NO"].count().sort_values(ascending=False).plot(kind='bar')
plt.title("Athletes from each country")
plt.show()
    """,
    """
import pandas as pd
import plotly.express as px

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Scatter plot of the unique number of athletes in each country, color-coded by sport
na_st = df.groupby(['Nationality', 'Sport'])['Name'].count().reset_index().rename(columns={'Name': 'NameCount'})
fig = px.scatter(na_st, x='Nationality', y='NameCount', color='Sport', size='NameCount', title='Unique number of Athletes in each country')
fig.update_layout(xaxis={'categoryorder': 'total descending'})
fig.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of total earnings per country
df.groupby("Nationality")["earnings ($ million)"].sum().plot(kind="bar")
plt.title("Total Earning as per Country")
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Line plot of total earnings over time by athletes
plt.plot(df.groupby("Year")["earnings ($ million)"].sum())
plt.xlabel('Year')
plt.ylabel('earnings ($ million)')
plt.title('Total Earnings over time by Athletes')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of total earnings per sport
df.groupby("Sport")["earnings ($ million)"].sum().sort_values().plot(kind="bar")
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of the top 5 highest earning athletes
df.groupby("Name")["earnings ($ million)"].sum().sort_values(ascending=False).head().plot(kind="bar")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Line plot of Floyd Mayweather's earnings over the years
floyd_mayweather = df[df['Name'] == 'Floyd Mayweather']
sns.relplot(x='Year', y='earnings ($ million)', kind='line', data=floyd_mayweather)
plt.title("Floyd Mayweather's Earnings")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of Floyd Mayweather's earnings over the years
floyd_mayweather = df[df['Name'] == 'Floyd Mayweather']
sns.catplot(x='Year', y='earnings ($ million)', kind='bar', data=floyd_mayweather)
plt.title("Floyd Mayweather's Earnings")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Line plot of Tiger Woods's earnings over the years
tiger_woods = df[df['Name'] == 'Tiger Woods']
sns.relplot(x='Year', y='earnings ($ million)', kind='line', data=tiger_woods)
plt.title("Tiger Woods's Earnings")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of Tiger Woods's earnings over the years
tiger_woods = df[df['Name'] == 'Tiger Woods']
sns.catplot(x='Year', y='earnings ($ million)', kind='bar', data=tiger_woods)
plt.title("Tiger Woods's Earnings")
plt.xticks(rotation=90)
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of total earnings per sport for athletes from the USA
usa = df[df['Nationality'] == 'USA']
usa.groupby('Sport')["earnings ($ million)"].sum().plot(kind="bar")
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of total earnings per sport for athletes from countries other than the USA
remaining_countries = df[df['Nationality'] != 'USA']
remaining_countries.groupby('Sport')['earnings ($ million)'].sum().sort_values(ascending=False).plot(kind='bar')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Bar plot of the earnings of the richest soccer players globally
remaining_countries[remaining_countries['Sport'] == 'Soccer'].groupby('Name')["earnings ($ million)"].sum().sort_values(ascending=False).plot(kind="bar")
plt.ylabel('earnings ($ million)')
plt.title('Soccer Players VS Earnings')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Read the CSV file
df = pd.read_csv('/kaggle/input/forbes-highest-paid-athletes-19902019/Forbes Richest Atheletes (Forbes Richest Athletes 1990-2020).csv')

# Line plot of the relationship between rank and earnings
df.groupby("Current Rank")["earnings ($ million)"].sum().plot()
plt.ylabel('Earnings')
plt.title("Rank VS Earnings")
plt.show()
    """,
    """
# Import needed Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read the CSV files
df_Departures = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Departures.csv")
df_Passengers = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv")

# Drop rows with missing carrier values
df_Departures = df_Departures.dropna()
df_Passengers = df_Passengers.dropna()

# Concatenate the dataframes
full_data = pd.concat([df_Departures, df_Passengers], axis=0)

# Count plot of the distribution of flight types (Departures and Passengers)
sns.countplot(x='type', data=full_data)
plt.title('Distribution Of Type', fontsize=20, color="black")
plt.grid(True)
plt.xticks(rotation=60)
plt.show()
    """,
    """
# Import needed Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read the CSV files
df_Departures = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Departures.csv")
df_Passengers = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv")

# Drop rows with missing carrier values
df_Departures = df_Departures.dropna()
df_Passengers = df_Passengers.dropna()

# Concatenate the dataframes
full_data = pd.concat([df_Departures, df_Passengers], axis=0)

# Relational plot of total flights by year
plt.figure(figsize=(10, 6))
sns.relplot(x='Year', y='Total', data=full_data, color='deepskyblue')
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year')
plt.xticks(rotation=45)
plt.show()
    """,
    """
# Import needed Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read the CSV files
df_Departures = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Departures.csv")
df_Passengers = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv")

# Drop rows with missing carrier values
df_Departures = df_Departures.dropna()
df_Passengers = df_Passengers.dropna()

# Concatenate the dataframes
full_data = pd.concat([df_Departures, df_Passengers], axis=0)

# Relational plot of total flights by year with carrier group hue
plt.figure(figsize=(10, 6))
sns.relplot(x='Year', y='Total', hue='carriergroup', data=full_data)
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year with Carrier Group Hue')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()
    """,
    """
# Import needed Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read the CSV files
df_Departures = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Departures.csv")
df_Passengers = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv")

# Drop rows with missing carrier values
df_Departures = df_Departures.dropna()
df_Passengers = df_Passengers.dropna()

# Concatenate the dataframes
full_data = pd.concat([df_Departures, df_Passengers], axis=0)

# Bar plot of total flights by year with type hue
plt.figure(figsize=(10, 6))
sns.barplot(x='Year', y='Total', hue='type', data=full_data)
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year with Type Hue')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()
    """,
    """
# Import needed Libraries 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

# Read the CSV files
df_Departures = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Departures.csv")
df_Passengers = pd.read_csv("/kaggle/input/us-international-air-traffic-data/International_Report_Passengers.csv")

# Drop rows with missing carrier values
df_Departures = df_Departures.dropna()
df_Passengers = df_Passengers.dropna()

# Concatenate the dataframes
full_data = pd.concat([df_Departures, df_Passengers], axis=0)

# Line plot of scheduled and chartered flights per year
plt.figure(figsize=(10, 6))
sns.lineplot(x='Year', y='Scheduled', data=full_data, label='Scheduled Flights')
sns.lineplot(x='Year', y='Charter', data=full_data, label='Chartered Flights')
plt.title('Total Flights per Year')
plt.xlabel('Year')
plt.ylabel('Total Flights (in Million)')
plt.xticks(rotation=45)
plt.grid(True)
plt.legend()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Data preprocessing
data.rename(columns={'Position ': 'Position'}, inplace=True)
data = data.dropna(axis=0, subset="Age")
data = data.drop(columns=[
    'Yearly bonus + stocks in EUR',
    'Annual brutto salary (without bonus and stocks) one year ago. Only answer if staying in the same country',
    'Annual bonus+stocks one year ago. Only answer if staying in same country',
    'Have you been forced to have a shorter working week (Kurzarbeit)? If yes, how many hours per week',
    'Have you received additional monetary support from your employer due to Work From Home? If yes, how much in 2020 in EUR'
])
data['Your main technology / programming language'] = data['Your main technology / programming language'].fillna('brak odp')
data['Other technologies/programming languages you use often'] = data['Other technologies/programming languages you use often'].fillna('brak odp')
data["Total years of experience"] = data["Total years of experience"].str.replace(",", ".").replace({
    "1 (as QA Engineer) / 11 in total": 11, 
    "15. thereof 8 as CTO": 15, 
    "6 (not as a data scientist. but as a lab scientist)": 6, 
    "less than year": 1
}).astype(float)
data["Years of experience in Germany"] = data["Years of experience in Germany"].str.replace(",", ".")
data["Years of experience in Germany"] = pd.to_numeric(data["Years of experience in Germany"], errors='coerce')
data["Number of vacation days"] = pd.to_numeric(data["Number of vacation days"], errors='coerce')

# KDE plot of the distribution of total years of experience
sns.kdeplot(data=data, x="Total years of experience")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Scatter plot of age vs total years of experience, colored by gender
sns.scatterplot(data=data, x="Age", y="Total years of experience", hue="Gender", alpha=0.8)
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Box plot of gender vs age at market entry
data["Wiek wejścia na rynek"] = data["Age"] - data["Total years of experience"]
sns.boxplot(data=data[data['Gender'] != 'Diverse'], x="Gender", y="Wiek wejścia na rynek")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Histograms of seniority levels for males and females
data['Seniority level'] = data['Seniority level'].replace(data['Seniority level'].value_counts()[data['Seniority level'].value_counts() < 4].index, 'other')
fig, axes = plt.subplots(1, 2, figsize=(15, 5), sharey=True)
sns.histplot(ax=axes[0], data=data[data['Gender'] == 'Male'], x='Seniority level')
axes[0].set_title('Men')
sns.histplot(ax=axes[1], data=data[data['Gender'] == 'Female'], x='Seniority level')
axes[1].set_title('Female')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Box plot of seniority level vs age
sns.boxplot(data=data, x="Seniority level", y="Age", order=['Junior', 'Middle', 'Senior', 'Lead', 'Head', 'other'])
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Histogram of seniority level vs city
data['Seniority level'] = pd.Categorical(data['Seniority level'], ['Junior', 'Middle', 'Senior', 'Lead', 'Head', 'other'])
data['City'] = data['City'].replace(data['City'].value_counts()[data['City'].value_counts() < 10].index, 'other')
sns.histplot(data=data, x='Seniority level', hue='City')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Box plot of gender vs yearly brutto salary
sns.boxplot(data=data[data['Gender'] != 'Diverse'], x="Gender", y="Yearly brutto salary (without bonus and stocks) in EUR")
plt.title("Zarobki a płeć")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from fuzzywuzzy import process

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Data preprocessing for main technology
data['Your main technology / programming language'] = data['Your main technology / programming language'].str.lower().str.strip()
for x in data['Your main technology / programming language'].value_counts().index[:20]:
    data['Your main technology / programming language'] = process.extractOne(x, data['Your main technology / programming language'])[0]

data['Your main technology / programming language'] = data['Your main technology / programming language'].replace(
    data['Your main technology / programming language'].value_counts()[data['Your main technology / programming language'].value_counts() < 10].index, 
    'other'
)
sns.boxplot(data=data, y='Your main technology / programming language', x='Yearly brutto salary (without bonus and stocks) in EUR', showfliers=False)
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Scatter plot of years of work abroad vs yearly brutto salary
data["Years of experience in Germany"] = pd.to_numeric(data["Years of experience in Germany"], errors="coerce")
data["Lata pracy za granicą"] = data["Total years of experience"] - data["Years of experience in Germany"]
data.loc[data["Lata pracy za granicą"] < 0, "Lata pracy za granicą"] = pd.NA
sns.scatterplot(data=data, x="Lata pracy za granicą", y="Yearly brutto salary (without bonus and stocks) in EUR", alpha=0.4)
plt.title("Lata pracy za granicą a roczne zarobki w euro")
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Load the dataset
data = pd.read_csv('/kaggle/input/IT Salary Survey EU  2020.csv')

# Box plot of company size vs yearly brutto salary
sns.boxplot(data=data, x="Company size", y="Yearly brutto salary (without bonus and stocks) in EUR", order=["up to 10", "11-50", "51-100", "101-1000", "1000+"])
plt.title("Wielkość firmy a roczna wysokość zarobków")
plt.show()
    """
]

In [6]:
not_std_graphs = [
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

# Load the iris dataset
iris = sns.load_dataset('iris')

# Create a new column for sepal area
iris['sepal_area'] = iris['sepal_length'] * iris['sepal_width']

# Scatter plot of sepal length vs sepal width
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sepal_length', y='sepal_width', hue='species', data=iris)

# Adding title and labels
plt.title('Scatter Plot of Sepal Length vs Sepal Width Across Species')
plt.xlabel('Sepal Length (cm)')
plt.ylabel('Sepal Width (cm)')

# Show plot
plt.show()
    """,
    """
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd

titanic = sns.load_dataset('titanic')

titanic['age'] = titanic['age'].fillna(titanic['age'].median())

bins = [0, 12, 18, 35, 60, 100]
labels = ['Child', 'Teenager', 'Adult', 'Middle-Aged', 'Senior']
titanic['age_group'] = pd.cut(titanic['age'], bins=bins, labels=labels)

survival_rate = titanic.groupby('pclass')['survived'].mean().reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='pclass', y='survived', data=survival_rate)

plt.title('Survival Rate by Passenger Class')
plt.xlabel('Passenger Class')
plt.ylabel('Survival Rate')

plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Create the figure and the first y-axis for Temperature
fig, ax1 = plt.subplots(figsize=(12, 6))

# Line plot for Temperature
ax1.plot(df['Month'], df['Temperature'], color='tab:red', marker='o', label='Temperature')
ax1.set_xlabel('Month')
ax1.set_ylabel('Temperature (°C)', color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')

# Create the second y-axis for Rainfall
ax2 = ax1.twinx()
ax2.bar(df['Month'], df['Rainfall'], color='tab:blue', alpha=0.6, label='Rainfall')
ax2.set_ylabel('Rainfall (mm)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Scatter plot for Humidity
ax1.scatter(df['Month'], df['Humidity'], color='tab:green', marker='s', s=100, label='Humidity', zorder=5)
ax1.set_ylim(0, 50)  # Adjusting y-limit to ensure visibility of scatter plot

# Add a legend
fig.tight_layout()  # To ensure no overlap of labels
fig.legend(loc='upper left', bbox_to_anchor=(0.1, 0.9))

plt.title('Monthly Temperature, Rainfall, and Humidity')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer

def create_pair_plot(csv_file):
  # Load the data
  df = pd.read_csv(csv_file)

  # Handle missing values
  imputer = SimpleImputer(strategy='mean')
  df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']] = imputer.fit_transform(
      df[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']])

  # Create the pair plot
  sns.pairplot(df, hue='species')
  plt.show()

create_pair_plot('penguins.csv')
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Write the sample data to a CSV file
with open('wine_quality.csv', 'w') as file:
    file.write(data)

# Load the wine quality dataset from the CSV file
wine = pd.read_csv('wine_quality.csv')

# Categorize alcohol levels
bins = [0, 8, 10, 12, 14, 20]  # Define bins for alcohol levels
labels = ['Very Low', 'Low', 'Moderate', 'High', 'Very High']  # Define labels for the bins
wine['alcohol_level'] = pd.cut(wine['alcohol'], bins=bins, labels=labels)

# Create histogram of wine quality
plt.figure(figsize=(10, 6))
sns.histplot(wine['quality'], bins=10, kde=False)

# Adding title and labels
plt.title('Histogram of Wine Quality')
plt.xlabel('Quality')
plt.ylabel('Frequency')

# Show plot
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the passengers dataset from the CSV file
df = pd.read_csv('passengers.csv')

# Convert 'year' and 'month' to a datetime index
df['date'] = pd.to_datetime(df['year'].astype(str) + '-' + df['month'])
df.set_index('date', inplace=True)

# Calculate cumulative passengers
df['cumulative_passengers'] = df['passengers'].cumsum()

# Create the line plot
plt.figure(figsize=(12, 6))
plt.plot(df.index, df['cumulative_passengers'], marker='o')
plt.xlabel('Date')
plt.ylabel('Cumulative Passengers')
plt.title('Cumulative Passengers Over Time')
plt.grid(True)
plt.xticks(rotation=45)
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the data
df = pd.read_csv('products.csv')

# Calculate total annual sales
df['Total Sales'] = df['Q1 Sales'] + df['Q2 Sales'] + df['Q3 Sales'] + df['Q4 Sales']

# Reshape the data for plotting
df_melted = df.melt(id_vars=['Product', 'Total Sales'], var_name='Quarter', value_name='Sales')

# Create the grouped bar chart with line plot overlay
plt.figure(figsize=(12, 6))

# Grouped bar chart
sns.barplot(x='Product', y='Sales', hue='Quarter', data=df_melted)

# Line plot for total sales
sns.lineplot(x='Product', y='Total Sales', data=df, color='red', marker='o')

plt.xlabel('Product')
plt.ylabel('Sales')
plt.title('Quarterly Sales by Product with Total Annual Sales')
plt.legend(title='Quarter')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the heart disease dataset from the CSV file
df = pd.read_csv('heart_disease.csv')

# Detect and remove outliers in cholesterol levels
q1 = df['chol'].quantile(0.25)
q3 = df['chol'].quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - 1.5 * iqr
upper_bound = q3 + 1.5 * iqr

# Filter out the outliers
df_filtered = df[(df['chol'] >= lower_bound) & (df['chol'] <= upper_bound)]

# Create the box plot of cholesterol levels by chest pain type
plt.figure(figsize=(12, 8))
sns.boxplot(x='cp', y='chol', data=df_filtered)

# Adding title and labels
plt.title('Box Plot of Cholesterol Levels by Chest Pain Type')
plt.xlabel('Chest Pain Type (cp)')
plt.ylabel('Cholesterol (mg/dL)')

# Show plot
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Load the data (might take some time due to download)
ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', dtype={'tconst': 'str', 'averageRating': 'float64', 'numVotes': 'int64'})
movies = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', dtype={'tconst': 'str', 'titleType': 'str', 'primaryTitle': 'str', 'originalTitle': 'str', 'isAdult': 'str', 'startYear': 'str', 'endYear': 'str', 'runtimeMinutes': 'str', 'genres': 'str'}, low_memory=False)

# Merge dataframes based on tconst
df = ratings.merge(movies[movies['titleType'] == 'movie'], on='tconst')

# Convert startYear to integer (handling potential errors)
df['startYear'] = pd.to_numeric(df['startYear'], errors='coerce')

# Group by startYear and calculate average rating
avg_ratings = df.groupby('startYear')['averageRating'].mean()

# Create the bar plot
plt.figure(figsize=(12, 6))
avg_ratings.plot(kind='bar')
plt.xlabel('Year')
plt.ylabel('Average Rating')
plt.title('Average Movie Ratings by Year')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.DataFrame(data1)

# Filter data for the years 2000, 2005, 2010, and 2015
years = [2000, 2005, 2010, 2015]
filtered_df1 = df1[df1['Year'].isin(years)]

# Sort data by suicide rates and select top countries
top_countries1 = filtered_df1.sort_values(by='Suicide rates', ascending=False).head(10)

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(top_countries1['Country'], top_countries1['Suicide rates'], color='skyblue')
plt.xlabel('Country')
plt.ylabel('Suicide Rates')
plt.title('Top Countries with the Highest Suicide Rates (2000, 2005, 2010, 2015)')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df2 = pd.DataFrame(data2)

# Filter data for the years 2000, 2005, 2010, and 2015
filtered_df2 = df2[df2['Year'].isin(years) & (df2['Sex'] == 'Male')]

# Sort data by suicide rates and select top countries
top_countries2 = filtered_df2.sort_values(by='Suicide rates', ascending=False).head(10)

# Plotting
plt.figure(figsize=(12, 6))
plt.bar(top_countries2['Country'], top_countries2['Suicide rates'], color='skyblue')
plt.xlabel('Country')
plt.ylabel('Suicide Rates')
plt.title('Top Countries with the Highest Suicide Rates for Males (2000, 2005, 2010, 2015)')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Aggregate the total numbers
totals = df[['Confirmed', 'Deaths', 'Recovered', 'Active']].sum()

# Plotting
plt.figure(figsize=(10, 6))
totals.plot(kind='bar', color=['skyblue', 'red', 'green', 'orange'])
plt.xlabel('Category')
plt.ylabel('Number of Cases')
plt.title('Total Number of Confirmed, Deaths, Recovered, and Active Cases Worldwide')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Group by WHO Region and sum confirmed cases
grouped_confirmed = df.groupby('WHO Region')['Confirmed'].sum()

# Bar plot
plt.figure(figsize=(10, 6))
grouped_confirmed.plot(kind='bar', color='skyblue')
plt.xlabel('WHO Region')
plt.ylabel('Confirmed Cases')
plt.title('Confirmed Cases Grouped by WHO Region')
plt.show()

# Pie chart
plt.figure(figsize=(8, 8))
grouped_confirmed.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'red', 'green'])
plt.ylabel('')
plt.title('Confirmed Cases Distribution by WHO Region')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Group by WHO Region and sum deaths
grouped_deaths = df.groupby('WHO Region')['Deaths'].sum()

# Bar plot
plt.figure(figsize=(10, 6))
grouped_deaths.plot(kind='bar', color='red')
plt.xlabel('WHO Region')
plt.ylabel('Deaths')
plt.title('Deaths Grouped by WHO Region')
plt.show()

# Pie chart
plt.figure(figsize=(8, 8))
grouped_deaths.plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'red', 'green'])
plt.ylabel('')
plt.title('Deaths Distribution by WHO Region')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Group by WHO Region and sum recovered and active cases
grouped_recovered = df.groupby('WHO Region')['Recovered'].sum()
grouped_active = df.groupby('WHO Region')['Active'].sum()

# Combine into a single dataframe for plotting
grouped_cases = pd.DataFrame({'Recovered': grouped_recovered, 'Active': grouped_active})

# Plotting
grouped_cases.plot(kind='bar', figsize=(12, 6), color=['green', 'orange'])
plt.xlabel('WHO Region')
plt.ylabel('Number of Cases')
plt.title('Recovered and Active Cases Grouped by WHO Region')
plt.legend(title='Case Type')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(datapoints)

# Select columns to sample
sampled_columns = ['Life expectancy', 'Adult Mortality', 'Infant deaths', 'Alcohol', 'BMI']

# Plotting histograms for each sampled column
df[sampled_columns].hist(bins=10, figsize=(15, 10), layout=(2, 3))
plt.suptitle('Distribution of Sampled Columns')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Plotting the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Sample numerical columns for scatter and density plots
numerical_columns = ['Life expectancy', 'Adult Mortality', 'Alcohol', 'BMI']

# Pairplot to show scatter plots and density plots
sns.pairplot(df[numerical_columns], diag_kind='kde')
plt.suptitle('Scatter and Density Plots of Numerical Features')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Plotting the pie chart for the target variable (output)
output_counts = df['output'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(output_counts, labels=output_counts.index, autopct='%1.1f%%', colors=['skyblue', 'salmon'], startangle=90)
plt.title('Distribution of Heart Attack Diagnosis (output)')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Binary variables
binary_vars = ['sex', 'fbs', 'exng']

# Plotting pie charts for each binary variable
plt.figure(figsize=(15, 5))
for i, var in enumerate(binary_vars):
    plt.subplot(1, 3, i + 1)
    counts = df[var].value_counts()
    plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90)
    plt.title(f'Distribution of {var}')

plt.tight_layout()
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Numerical and categorical variables
numerical_vars = ['age', 'trtbps', 'chol', 'thalachh', 'oldpeak']
categorical_vars = ['cp', 'restecg', 'slp', 'caa', 'thall']

# Plotting histograms for numerical variables
df[numerical_vars].hist(bins=10, figsize=(15, 10))
plt.suptitle('Distribution of Numerical Features')
plt.show()

# Plotting bar plots for categorical variables
plt.figure(figsize=(15, 10))
for i, var in enumerate(categorical_vars):
    plt.subplot(2, 3, i + 1)
    counts = df[var].value_counts()
    plt.bar(counts.index, counts.values, color='skyblue')
    plt.title(f'Distribution of {var}')
    plt.xlabel(var)
    plt.ylabel('Count')

plt.tight_layout()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Plotting the correlation matrix heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Pie chart for placement status
status_counts = df['status'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(status_counts, labels=status_counts.index, autopct='%1.1f%%', colors=['skyblue', 'salmon'], startangle=90)
plt.title('Distribution of Placement Status')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(file)

# Pie charts for gender, specialization, and work experience
fig, axes = plt.subplots(1, 3, figsize=(20, 6))

# Gender distribution
gender_counts = df['gender'].value_counts()
axes[0].pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90)
axes[0].set_title('Distribution of Gender')

# Specialization distribution
specialisation_counts = df['specialisation'].value_counts()
axes[1].pie(specialisation_counts, labels=specialisation_counts.index, autopct='%1.1f%%', startangle=90)
axes[1].set_title('Distribution of Specialisation')

# Work experience distribution
workex_counts = df['workex'].value_counts()
axes[2].pie(workex_counts, labels=workex_counts.index, autopct='%1.1f%%', startangle=90)
axes[2].set_title('Distribution of Work Experience')

plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Select numerical and categorical features
numerical_features = ['ssc_p', 'hsc_p', 'degree_p', 'etest_p', 'mba_p', 'salary', 'avg_p']
categorical_features = ['gender', 'ssc_b', 'hsc_b', 'hsc_s', 'degree_t', 'workex', 'specialisation', 'status']

# Plotting histograms for numerical features
df[numerical_features].hist(bins=10, figsize=(15, 10))
plt.suptitle('Distribution of Numerical Features')
plt.show()

# Plotting bar plots for categorical features
fig, axes = plt.subplots(len(categorical_features), 1, figsize=(10, 20))
for i, feature in enumerate(categorical_features):
    counts = df[feature].value_counts()
    axes[i].bar(counts.index, counts.values, color='skyblue')
    axes[i].set_title(f'Distribution of {feature}')
    axes[i].set_xlabel(feature)
    axes[i].set_ylabel('Count')
    axes[i].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Calculate the correlation matrix
correlation_matrix = df.corr()

# Plotting the correlation matrix heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Matrix Heatmap')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Plotting the histogram for movie ratings
plt.figure(figsize=(10, 6))
plt.hist(df['rating'], bins=5, edgecolor='black', color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Movie Ratings')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Find the top-10 most rated movies
top_movies = df['movieId'].value_counts().head(10).index
top_movies_df = df[df['movieId'].isin(top_movies)]

# Plotting the boxplot for the top-10 most rated movies
plt.figure(figsize=(12, 8))
top_movies_df.boxplot(column='rating', by='movieId', grid=False)
plt.xlabel('Movie ID')
plt.ylabel('Rating')
plt.title('Distribution of Ratings for Top-10 Most Rated Movies')
plt.suptitle('')  # Suppress the automatic title to avoid duplication
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Plotting the histogram for movie ratings
plt.figure(figsize=(10, 6))
df['rating'].value_counts().plot(kind='bar', color='skyblue')
plt.xlabel('Rating')
plt.ylabel('Frequency')
plt.title('Distribution of Movie Ratings')
plt.show()
    """,
    """
import seaborn as sns

# Plotting the countplot for release years
plt.figure(figsize=(10, 6))
sns.countplot(y='release_year', data=df, palette='viridis', order=df['release_year'].value_counts().index)
plt.xlabel('Count')
plt.ylabel('Release Year')
plt.title('Distribution of Release Years of Netflix Titles')
plt.show()
    """,
    """
df['director'] = df['director'].replace(['Director1', 'Director2', 'Director3', 'Director4', 'Director5'], 
                                         ['Director1', 'Director1', 'Director2', 'Director2', 'Director3'])

# Plotting the bar plot for top 10 directors
top_directors = df['director'].value_counts().head(10)
plt.figure(figsize=(10, 6))
top_directors.plot(kind='bar', color='skyblue')
plt.xlabel('Director')
plt.ylabel('Number of Titles')
plt.title('Top 10 Directors with Most Titles on Netflix')
plt.show()
    """,
    """
# Plotting the countplot for movies and TV shows
plt.figure(figsize=(10, 6))
sns.countplot(x='type', data=df, palette='viridis')
plt.xlabel('Type')
plt.ylabel('Count')
plt.title('Distribution of Movies and TV Shows on Netflix')
plt.show()
    """,
    """
plt.figure(figsize=(10, 6))
sns.countplot(y='rating', data=df, palette='viridis', order=df['rating'].value_counts().index)
plt.xlabel('Count')
plt.ylabel('Rating')
plt.title('Distribution of Ratings of Netflix Titles')
plt.show()
    """,
    """
genres = df['genre'].str.split(', ').sum()
top_genres = pd.Series(Counter(genres)).sort_values(ascending=False).head(10)

# Plotting the bar plot for top 10 genres
plt.figure(figsize=(10, 6))
top_genres.plot(kind='bar', color='skyblue')
plt.xlabel('Genre')
plt.ylabel('Number of Titles')
plt.title('Top 10 Genres on Netflix')
plt.show()
    """,
    """
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Split the data into training and testing datasets
# Assuming we use the first 80% of the data as training data
train_size = int(len(df) * 0.8)
train_df = df[:train_size]

# Plotting the relationship between the opening and closing values
plt.figure(figsize=(10, 6))
plt.scatter(train_df['Open'], train_df['Close'], alpha=0.5, color='blue')
plt.xlabel('Opening Value')
plt.ylabel('Closing Value')
plt.title('Relationship between Opening and Closing Values of Tesla Stock (Training Data)')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df1 = pd.DataFrame(data1)

df2 = pd.DataFrame(data2)

# Merge the datasets on NOC
merged_df = pd.merge(df1, df2, on='NOC')

# Calculate the number of participants per region
participant_counts = merged_df['region'].value_counts().head(10)

# Plotting the bar plot for the top 10 participating countries
plt.figure(figsize=(12, 8))
participant_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Country')
plt.ylabel('Number of Participants')
plt.title('Top 10 Participating Countries in the Olympics (Summer and Winter)')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Bar plot of the number of athletes from each country
country_counts = df['Nationality'].value_counts()
plt.figure(figsize=(10, 6))
country_counts.plot(kind='bar', color='skyblue')
plt.xlabel('Country')
plt.ylabel('Number of Athletes')
plt.title('Number of Athletes from Each Country')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Scatter plot of the unique number of athletes in each country, color-coded by sport
plt.figure(figsize=(10, 6))
for sport in df['Sport'].unique():
    subset = df[df['Sport'] == sport]
    plt.scatter(subset['Nationality'], subset['Name'], label=sport)
plt.xlabel('Country')
plt.ylabel('Athletes')
plt.title('Unique Number of Athletes in Each Country, Color-coded by Sport')
plt.legend()
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Bar plot of total earnings per country
earnings_per_country = df.groupby('Nationality')['earnings ($ million)'].sum()
plt.figure(figsize=(10, 6))
earnings_per_country.plot(kind='bar', color='skyblue')
plt.xlabel('Country')
plt.ylabel('Total Earnings ($ million)')
plt.title('Total Earnings per Country')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Line plot of total earnings over time by athletes
earnings_over_time = df.groupby('Year')['earnings ($ million)'].sum()
plt.figure(figsize=(10, 6))
earnings_over_time.plot(kind='line', marker='o')
plt.xlabel('Year')
plt.ylabel('Total Earnings ($ million)')
plt.title('Total Earnings Over Time by Athletes')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Bar plot of total earnings per sport
earnings_per_sport = df.groupby('Sport')['earnings ($ million)'].sum()
plt.figure(figsize=(10, 6))
earnings_per_sport.plot(kind='bar', color='skyblue')
plt.xlabel('Sport')
plt.ylabel('Total Earnings ($ million)')
plt.title('Total Earnings per Sport')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Bar plot of the top 5 highest earning athletes
top_5_athletes = df.nlargest(5, 'earnings ($ million)')
plt.figure(figsize=(10, 6))
plt.bar(top_5_athletes['Name'], top_5_athletes['earnings ($ million)'], color='skyblue')
plt.xlabel('Athlete')
plt.ylabel('Earnings ($ million)')
plt.title('Top 5 Highest Earning Athletes')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for Floyd Mayweather
floyd_data = df[df['Name'] == 'Floyd Mayweather']

# Line plot of Floyd Mayweather's earnings over the years
plt.figure(figsize=(10, 6))
plt.plot(floyd_data['Year'], floyd_data['earnings ($ million)'], marker='o', color='skyblue')
plt.xlabel('Year')
plt.ylabel('Earnings ($ million)')
plt.title('Floyd Mayweather\'s Earnings Over the Years')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

# Filter data for Floyd Mayweather
floyd_data = df[df['Name'] == 'Floyd Mayweather']

# Bar plot of Floyd Mayweather's earnings over the years
plt.figure(figsize=(10, 6))
plt.bar(floyd_data['Year'], floyd_data['earnings ($ million)'], color='skyblue')
plt.xlabel('Year')
plt.ylabel('Earnings ($ million)')
plt.title('Floyd Mayweather\'s Earnings Over the Years')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for Tiger Woods
tiger_data = df[df['Name'] == 'Tiger Woods']

# Line plot of Tiger Woods's earnings over the years
plt.figure(figsize=(10, 6))
plt.plot(tiger_data['Year'], tiger_data['earnings ($ million)'], marker='o', color='green')
plt.xlabel('Year')
plt.ylabel('Earnings ($ million)')
plt.title('Tiger Woods\'s Earnings Over the Years')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for Tiger Woods
tiger_data = df[df['Name'] == 'Tiger Woods']

# Bar plot of Tiger Woods's earnings over the years
plt.figure(figsize=(10, 6))
plt.bar(tiger_data['Year'], tiger_data['earnings ($ million)'], color='green')
plt.xlabel('Year')
plt.ylabel('Earnings ($ million)')
plt.title('Tiger Woods\'s Earnings Over the Years')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for athletes from the USA
usa_data = df[df['Nationality'] == 'USA']

# Bar plot of total earnings per sport for athletes from the USA
earnings_per_sport_usa = usa_data.groupby('Sport')['earnings ($ million)'].sum()
plt.figure(figsize=(10, 6))
earnings_per_sport_usa.plot(kind='bar', color='skyblue')
plt.xlabel('Sport')
plt.ylabel('Total Earnings ($ million)')
plt.title('Total Earnings per Sport for Athletes from the USA')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for athletes from countries other than the USA
non_usa_data = df[df['Nationality'] != 'USA']

# Bar plot of total earnings per sport for athletes from countries other than the USA
earnings_per_sport_non_usa = non_usa_data.groupby('Sport')['earnings ($ million)'].sum()
plt.figure(figsize=(10, 6))
earnings_per_sport_non_usa.plot(kind='bar', color='salmon')
plt.xlabel('Sport')
plt.ylabel('Total Earnings ($ million)')
plt.title('Total Earnings per Sport for Athletes from Countries Other Than the USA')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Filter data for soccer players
soccer_players = df[df['Sport'] == 'Soccer']

# Bar plot of the earnings of the richest soccer players globally
plt.figure(figsize=(10, 6))
plt.bar(soccer_players['Name'], soccer_players['earnings ($ million)'], color='skyblue')
plt.xlabel('Soccer Player')
plt.ylabel('Earnings ($ million)')
plt.title('Earnings of the Richest Soccer Players Globally')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Line plot of the relationship between rank and earnings
plt.figure(figsize=(10, 6))
plt.plot(df['Current Rank'], df['earnings ($ million)'], marker='o', color='skyblue')
plt.xlabel('Current Rank')
plt.ylabel('Earnings ($ million)')
plt.title('Relationship Between Rank and Earnings')
plt.gca().invert_xaxis()  # Invert x-axis to show rank from highest to lowest
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Plotting the count plot for flight types
plt.figure(figsize=(10, 6))
sns.countplot(data=df, x='type')
plt.xlabel('Flight Type')
plt.ylabel('Count')
plt.title('Distribution of Flight Types (Departures and Passengers)')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Relational plot of total flights by year
plt.figure(figsize=(10, 6))
sns.relplot(data=df, x='Year', y='Total', kind='line')
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Relational plot of total flights by year with carrier group hue
plt.figure(figsize=(10, 6))
sns.relplot(data=df, x='Year', y='Total', hue='carriergroup', kind='line')
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year with Carrier Group Hue')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Bar plot of total flights by year with type hue
plt.figure(figsize=(10, 6))
sns.barplot(data=df, x='Year', y='Total', hue='type')
plt.xlabel('Year')
plt.ylabel('Total Flights')
plt.title('Total Flights by Year with Type Hue')
plt.show()
    """,
    """
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Group data by year
grouped_df = df.groupby('Year').sum()

# Line plot of scheduled and chartered flights per year
plt.figure(figsize=(10, 6))
plt.plot(grouped_df.index, grouped_df['Scheduled'], label='Scheduled', marker='o')
plt.plot(grouped_df.index, grouped_df['Charter'], label='Charter', marker='o')
plt.xlabel('Year')
plt.ylabel('Number of Flights')
plt.title('Scheduled and Chartered Flights per Year')
plt.legend()
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# KDE plot of the distribution of total years of experience
plt.figure(figsize=(10, 6))
sns.kdeplot(df['Total years of experience'], shade=True)
plt.xlabel('Total Years of Experience')
plt.title('Distribution of Total Years of Experience')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Scatter plot of age vs total years of experience, colored by gender
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Age', y='Total years of experience', hue='Gender')
plt.xlabel('Age')
plt.ylabel('Total Years of Experience')
plt.title('Age vs Total Years of Experience')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Calculate age at market entry
df['Age at Market Entry'] = df['Age'] - df['Total years of experience']

# Box plot of gender vs age at market entry
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Gender', y='Age at Market Entry')
plt.xlabel('Gender')
plt.ylabel('Age at Market Entry')
plt.title('Gender vs Age at Market Entry')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Histograms of seniority levels for males and females
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='Seniority level', hue='Gender', multiple='dodge', shrink=.8)
plt.xlabel('Seniority Level')
plt.title('Seniority Levels for Males and Females')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Box plot of seniority level vs age
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Seniority level', y='Age')
plt.xlabel('Seniority Level')
plt.ylabel('Age')
plt.title('Seniority Level vs Age')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Hist plot of seniority level vs city
plt.figure(figsize=(12, 6))
sns.histplot(data=df, x='City', hue='Seniority level', multiple='stack')
plt.xlabel('City')
plt.title('Seniority Level vs City')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Box plot of gender vs yearly brutto salary
plt.figure(figsize=(10, 6))
sns.boxplot(data=df, x='Gender', y='Yearly brutto salary (without bonus and stocks) in EUR')
plt.xlabel('Gender')
plt.ylabel('Yearly Brutto Salary (EUR)')
plt.title('Gender vs Yearly Brutto Salary')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Scatter plot of total years of experience vs yearly brutto salary, colored by gender
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df, x='Total years of experience', y='Yearly brutto salary (without bonus and stocks) in EUR', hue='Gender')
plt.xlabel('Total Years of Experience')
plt.ylabel('Yearly Brutto Salary (EUR)')
plt.title('Total Years of Experience vs Yearly Brutto Salary')
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Box plot of main technology vs yearly brutto salary
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Your main technology / programming language', y='Yearly brutto salary (without bonus and stocks) in EUR')
plt.xlabel('Main Technology / Programming Language')
plt.ylabel('Yearly Brutto Salary (EUR)')
plt.title('Main Technology vs Yearly Brutto Salary')
plt.xticks(rotation=45)
plt.show()
    """,
    """
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

df = pd.DataFrame(data)

# Box plot of company size vs yearly brutto salary
plt.figure(figsize=(12, 6))
sns.boxplot(data=df, x='Company size', y='Yearly brutto salary (without bonus and stocks) in EUR')
plt.xlabel('Company Size')
plt.ylabel('Yearly Brutto Salary (EUR)')
plt.title('Company Size vs Yearly Brutto Salary')
plt.show()
    """
]

In [7]:
print(len(not_std_graphs) == len(gold_std_graphs) == len(scores) == len(data_descriptions) == len(graph_descriptions))
print(len(not_std_graphs))
print(len(gold_std_graphs))
print(len(scores))
print(len(data_descriptions))
print(len(graph_descriptions))

True
65
65
65
65
65


In [8]:
NUM_THREADS = 4

In [9]:
# should be a list of dspy.Examples with field: prediction, gold_standard, and score
def makeMetricExamples(gold_standard_graphs, predicted_graphs, scores):
    exampleMetricList = []
    for i in range(len(gold_standard_graphs)):
        gold_standard = gold_standard_graphs[i]
        prediction_code = predicted_graphs[i]
        score = scores[i]
        metricObj = dspy.Example(prediction_code=prediction_code, gold_standard=gold_standard, score=score).with_inputs("prediction_code", "gold_standard")
        exampleMetricList.append(metricObj)
    return exampleMetricList

def get_first_number(text):
    match = re.search(r'\d+', text)
    if match:
        return int(match.group())
    return None

class GraphEvaluatorSignature(dspy.Signature):
    """Assess the quality of a graph determined by how well it matches the gold_standard graph."""
    prediction_code = dspy.InputField()
    gold_standard = dspy.InputField()
    output_score = dspy.OutputField(desc="A 1-3 rating of how well prediction_code graph matches the gold_standard graph")

class GraphEvaluator(dspy.Module):
    def __init__(self):
        super().__init__()
        self.score_code = dspy.ChainOfThought(GraphEvaluatorSignature)
        
    def forward(self, prediction_code, gold_standard):
        score = self.score_code(prediction_code=prediction_code, gold_standard=gold_standard)
        score_num = get_first_number(score.output_score)
        # Assert that output_score is a number from 0 to 3
        if not score:
            print(score)
            print(f"as: {score_num}")
        dspy.Assert(
            0 <= score_num <= 3,
            "Output score must be from 1 (worst rating) to 3 (best rating)"
        )
        
        return score_num
        
def accuracy_metric(example, pred, trace=None):
    return -abs(int(example.score) - pred)

In [10]:
scores = np.array(scores)
# Split the data with stratification
train_indices, test_indices = train_test_split(
    np.arange(len(scores)),
    stratify=scores,
    test_size=0.3,
    random_state=42
)

# Ensure the distribution is correct
train_scores = scores[train_indices]
test_scores = scores[test_indices]

# Count occurrences in training set
train_count_1 = np.sum(train_scores == 1)
train_count_2 = np.sum(train_scores == 2)
train_count_3 = np.sum(train_scores == 3)

# Count occurrences in testing set
test_count_1 = np.sum(test_scores == 1)
test_count_2 = np.sum(test_scores == 2)
test_count_3 = np.sum(test_scores == 3)

(train_indices, train_count_1, train_count_2, train_count_3), (test_indices, test_count_1, test_count_2, test_count_3)
scores = scores.tolist()

In [11]:
train_indices

array([14,  1, 22, 26, 38,  9, 18, 57, 11, 35, 10,  5, 52,  0, 19, 21, 54,
       61, 43, 40, 36,  7, 13, 25, 41,  3, 55, 59,  8, 37, 31, 53, 60, 30,
        2, 46, 56, 15, 51, 50, 12,  6, 63, 28, 58])

In [12]:
test_indices

array([23, 27, 45, 16, 39,  4, 47, 49, 24, 33, 34, 29, 44, 48, 62, 42, 64,
       32, 20, 17])

In [13]:
metricExamples = makeMetricExamples(gold_std_graphs, not_std_graphs, scores)
trainset = [metricExamples[i] for i in train_indices]
testset = [metricExamples[i] for i in test_indices]

In [14]:
trainset[0]

Example({'prediction_code': "\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\ndf = pd.DataFrame(data)\n\n# Group by WHO Region and sum recovered and active cases\ngrouped_recovered = df.groupby('WHO Region')['Recovered'].sum()\ngrouped_active = df.groupby('WHO Region')['Active'].sum()\n\n# Combine into a single dataframe for plotting\ngrouped_cases = pd.DataFrame({'Recovered': grouped_recovered, 'Active': grouped_active})\n\n# Plotting\ngrouped_cases.plot(kind='bar', figsize=(12, 6), color=['green', 'orange'])\nplt.xlabel('WHO Region')\nplt.ylabel('Number of Cases')\nplt.title('Recovered and Active Cases Grouped by WHO Region')\nplt.legend(title='Case Type')\nplt.show()\n    ", 'gold_standard': "\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# Load the dataset\ndf = pd.read_csv('/kaggle/input/corona-virus-report/country_wise_latest.csv')\n\n# Grouping by WHO Region\nRegion = df.groupby(by='WHO Region').sum()\n\n# Bar plot for Recovered and Active Cases by WHO Region

In [15]:
# config = {
#     "max_bootstrapped_demos": 4,
#     "max_labeled_demos": 4,
#     "num_candidate_programs": 6,
#     "num_threads": 4
# }

# metric_teleprompter = BootstrapFewShotWithRandomSearch(
#     metric=accuracy_metric,
#     **config
# )

# # Compile the optimized program
# compiled_metric = metric_teleprompter.compile(GraphEvaluator(), trainset=trainset)

metric_teleprompter = LabeledFewShot(k=5)
compiled_metric = metric_teleprompter.compile(GraphEvaluator(), trainset=trainset)

In [15]:
evaluater = Evaluate(devset=testset, metric=accuracy_metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)

In [17]:
evaluater(compiled_metric)

Average Metric: -16 / 20  (-80.0): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:13<00:00,  1.45it/s]


-80.0

In [16]:
# with bootstrap
loaded_program = GraphEvaluator()
loaded_program.load(path="DSPy/compiled_metric")
evaluater(loaded_program)

Average Metric: -15 / 20  (-75.0): 100%|████████| 20/20 [00:09<00:00,  2.15it/s]


-75.0

In [63]:
# few shot
# metric_teleprompter = LabeledFewShot(k=5)
# compiled_fs_metric = metric_teleprompter.compile(GraphEvaluator(), trainset=trainset)
evaluater(compiled_fs_metric)

Average Metric: -17 / 20  (-85.0): 100%|████████| 20/20 [00:12<00:00,  1.57it/s]


-85.0

In [19]:
# zero shot:
compiled_zs_metric = GraphEvaluator()
evaluater(compiled_zs_metric)

Average Metric: -18 / 20  (-90.0): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:15<00:00,  1.31it/s]


-90.0

In [17]:
# THIS IS USING LOADED METRIC NOT COMPILED METRIC
def metric(example, pred, trace=None):
    data_description, graph_description, prediction_code = example.data_description, example.graph_description, pred

    # Use the compiled metric evaluator
    evaluation_result = loaded_program(prediction_code=prediction_code, gold_standard=example.graph)
    return abs(evaluation_result)

In [18]:
def makeDSPYExamples(graphs, data_descriptions, graph_descriptions):
    exampleList = []
    for i in range(len(graphs)):
        data_desc = data_descriptions[i]
        graph_desc = graph_descriptions[i]
        graph = graphs[i]
        exampleObj = dspy.Example(data_description=data_desc, graph_description=graph_desc, graph=graph).with_inputs("data_description", "graph_description")
        exampleList.append(exampleObj)
    return exampleList

class TextToDataManipulation(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_data_manipulation = dspy.ChainOfThought(TextToDataManipulationSignature)

    def forward(self, data_description, graph_description):
        data_manipulation_code = self.generate_data_manipulation(data_description=data_description, graph_description=graph_description)
        return data_manipulation_code

class TextToDataManipulationSignature(dspy.Signature):
    """Generate data manipulation code from data and target graph descriptions"""
    data_description = dspy.InputField()
    graph_description = dspy.InputField()
    data_manipulation_code = dspy.OutputField()

class TextToGraphCode(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_graph_code = dspy.ChainOfThought(TextToGraphCodeSignature)

    def forward(self, graph_description, data_description, data_manipulation_code):
        graph_code = self.generate_graph_code(graph_description=graph_description, data_description=data_description, data_manipulation_code=data_manipulation_code)
        return graph_code

class TextToGraphCodeSignature(dspy.Signature):
    """Generate graph code from a given description and data"""
    graph_description = dspy.InputField()
    data_description = dspy.InputField()
    data_manipulation_code = dspy.InputField()
    graph_code = dspy.OutputField()

class Analyzer(dspy.Module):
    def __init__(self):
        super().__init__()
        self.text_to_data_manipulation = TextToDataManipulation()
        self.text_to_graph_code = TextToGraphCode()

    def forward(self, graph_description, data_description):
        # data manipulation code
        data_manipulation_code_prediction = self.text_to_data_manipulation(data_description=data_description, graph_description=graph_description)
        
        # get the actual code from the Prediction object
        data_manipulation_code = data_manipulation_code_prediction.data_manipulation_code
        
        # graph code
        graph_code_prediction = self.text_to_graph_code(graph_description=graph_description, data_description=data_description, data_manipulation_code=data_manipulation_code)
        
        # get the actual graph code from the Prediction object
        graph_code = graph_code_prediction.graph_code
        
        # combine the data manipulation code and the graph code
        complete_code = f"{data_manipulation_code}\n{graph_code}"
        # print(complete_code)
        
        return complete_code

class AnalyzerSignature(dspy.Signature):
    """Generate python code to create a graph for the given description"""
    data_description = dspy.InputField()
    graph_description = dspy.InputField()
    complete_code = dspy.OutputField()


In [19]:
allExamples = makeDSPYExamples(gold_std_graphs, data_descriptions, graph_descriptions)
trainset = allExamples[:45]
testset = allExamples[45:]

In [17]:
teleprompter = LabeledFewShot(k=7)
compiled = teleprompter.compile(Analyzer(), trainset=trainset)

In [ ]:
llama = dspy.Together(model="meta-llama/Meta-Llama-3.1-405B-Instruct-Turbo")
dspy.configure(lm=llama)

In [21]:
# teleprompter = LabeledFewShot(k=7)
# compiled = teleprompter.compile(Analyzer(), trainset=trainset)
config = dict(max_bootstrapped_demos=4, max_labeled_demos=4, num_candidate_programs=6, num_threads=4)

teleprompter = BootstrapFewShotWithRandomSearch(
    metric=metric,
    **config
)

compiled = teleprompter.compile(Analyzer(), trainset=trainset)

Going to sample between 1 and 4 traces per predictor.
Will attempt to bootstrap 6 candidate sets.


Average Metric: 110 / 45  (244.4): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:02<00:00,  1.40s/it]


Score: 244.44 for set: [0, 0]
New best sscore: 244.44 for seed -3
Scores so far: [244.44]
Best score: 244.44


Average Metric: 111 / 45  (246.7): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:04<00:00,  1.43s/it]


Score: 246.67 for set: [4, 4]
New best sscore: 246.67 for seed -2
Scores so far: [244.44, 246.67]
Best score: 246.67


...

Score: 204.44 for set: [4, 4]
Scores so far: [244.44, 246.67, 231.11, 235.56, 233.33, 217.78, 257.78, 204.44]
Best score: 257.78
Average of max per entry across top 1 scores: 2.577777777777778
Average of max per entry across top 2 scores: 2.688888888888889
Average of max per entry across top 3 scores: 2.7333333333333334
Average of max per entry across top 5 scores: 2.7333333333333334
Average of max per entry across top 8 scores: 2.8222222222222224
Average of max per entry across top 9999 scores: 2.8222222222222224


  7%|██████████████▌                                                                                                                                                                                                           | 3/45 [00:16<03:54,  5.59s/it]


Bootstrapped 3 full traces after 4 examples in round 0.


Average Metric: 100 / 45  (222.2): 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45/45 [01:31<00:00,  2.04s/it]

Score: 222.22 for set: [4, 4]
Scores so far: [244.44, 246.67, 231.11, 235.56, 233.33, 217.78, 257.78, 204.44, 222.22]
Best score: 257.78
Average of max per entry across top 1 scores: 2.577777777777778
Average of max per entry across top 2 scores: 2.688888888888889
Average of max per entry across top 3 scores: 2.7333333333333334
Average of max per entry across top 5 scores: 2.7333333333333334
Average of max per entry across top 8 scores: 2.7777777777777777
Average of max per entry across top 9999 scores: 2.8222222222222224
9 candidate programs found.


In [20]:
evaluater = Evaluate(devset=testset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=0)
# evaluater(compiled)

In [24]:
# need to do on compiled
evaluater(Analyzer())

Average Metric: 46 / 20  (230.0): 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:29<00:00,  1.48s/it]


230.0

In [21]:
loaded_gen_GPT = Analyzer()
loaded_gen_GPT.load(path="DSPy/compiled_program-GPT")
evaluater(loaded_gen_GPT)

Average Metric: 47 / 20  (235.0): 100%|█████████| 20/20 [00:29<00:00,  1.48s/it]


235.0

In [23]:
# Initialize the analyzer module
analyzer = loaded_gen_GPT

data_description = data_descriptions[9]
graph_description = graph_descriptions[9]

# Generate the data manipulation code
data_manipulation_code = analyzer.text_to_data_manipulation(data_description=data_description, graph_description=graph_description)
print("Data Manipulation Code:")
print(data_manipulation_code)

data_manipulation_code = data_manipulation_code.data_manipulation_code

# Generate the graph code
graph_code = analyzer.text_to_graph_code(graph_description=graph_description, data_description=data_description, data_manipulation_code=data_manipulation_code)
print("\nGraph Code:")
print(graph_code)

# Generate the complete code
complete_code = analyzer(graph_description=graph_description, data_description=data_description)
print("\nComplete Code:")
print(complete_code)

Data Manipulation Code:
Prediction(
    rationale='generate the data manipulation code. We need to import the necessary libraries, read the CSV file, and then manipulate the data to prepare it for the graph. We need to filter the data for the specific years, group the data by country, and calculate the average suicide rate for each country. We can use the pandas library to achieve this.',
    data_manipulation_code="``` import pandas as pd import matplotlib.pyplot as plt # Read the CSV file df = pd.read_csv('suicide_rates.csv') # Filter the data for the specific years years = [2000, 2005, 2010, 2015] df_filtered = df[df['Year'].isin(years)] # Group the data by country and calculate the average suicide rate for each country df_grouped = df_filtered.groupby('Country')['Suicide rates'].mean().reset_index() # Get the top countries with the highest suicide rates df_top = df_grouped.nlargest(10, 'Suicide rates') # Prepare the data for the bar plot x = df_top['Country'] y = df_top['Suicide ra

In [24]:
llama.inspect_history(n=1)




Generate graph code from a given description and data

---

Follow the following format.

Graph Description: ${graph_description}

Data Description: ${data_description}

Data Manipulation Code: ${data_manipulation_code}

Reasoning: Let's think step by step in order to ${produce the graph_code}. We ...

Graph Code: ${graph_code}

---

Graph Description: Bar and pie charts showing the deaths grouped by WHO Regions.

Data Description: Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.

Data Manipulation Code: ``` import pandas as pd # Read the CSV file df = pd.read_csv('data.csv') # Group the deaths by WHO Regions and calculate the total deaths for each re

"\n\n\nGenerate graph code from a given description and data\n\n---\n\nFollow the following format.\n\nGraph Description: ${graph_description}\n\nData Description: ${data_description}\n\nData Manipulation Code: ${data_manipulation_code}\n\nReasoning: Let's think step by step in order to ${produce the graph_code}. We ...\n\nGraph Code: ${graph_code}\n\n---\n\nGraph Description: Bar and pie charts showing the deaths grouped by WHO Regions.\n\nData Description: Fields: Country/Region (string), Confirmed (integer), Deaths (integer), Recovered (integer), Active (integer), New cases (integer), New deaths (integer), New recovered (integer), Deaths / 100 Cases (float), Recovered / 100 Cases (float), Deaths / 100 Recovered (float), Confirmed last week (integer), 1 week change (integer), 1 week % increase (float), WHO Region (string). Format: CSV.\n\nData Manipulation Code: ``` import pandas as pd # Read the CSV file df = pd.read_csv('data.csv') # Group the deaths by WHO Regions and calculate th

In [30]:
llama.inspect_history(n=1)




Generate graph code from a given description and data

---

Follow the following format.

Graph Description: ${graph_description}

Data Description: ${data_description}

Data Manipulation Code: ${data_manipulation_code}

Reasoning: Let's think step by step in order to ${produce the graph_code}. We ...

Graph Code: ${graph_code}

---

Graph Description: Bar plot showing the top countries with the highest suicide rates for the years 2000, 2005, 2010, and 2015.

Data Description: Fields: Entity (string), Year (integer), Suicide rates (float), Crude suicide rates per gender (float), Country (string), Age standardized (boolean). Format: CSV. Coverage: 198 countries, 1950-2011

Data Manipulation Code: ``` import pandas as pd import matplotlib.pyplot as plt # Load the data from the CSV file data = pd.read_csv('suicide_rates.csv') # Filter the data for the specific years of interest years_of_interest = [2000, 2005, 2010, 2015] data_filtered = data[data['Year'].isin(years_of_interest)] # Gro

"\n\n\nGenerate graph code from a given description and data\n\n---\n\nFollow the following format.\n\nGraph Description: ${graph_description}\n\nData Description: ${data_description}\n\nData Manipulation Code: ${data_manipulation_code}\n\nReasoning: Let's think step by step in order to ${produce the graph_code}. We ...\n\nGraph Code: ${graph_code}\n\n---\n\nGraph Description: Bar plot showing the top countries with the highest suicide rates for the years 2000, 2005, 2010, and 2015.\n\nData Description: Fields: Entity (string), Year (integer), Suicide rates (float), Crude suicide rates per gender (float), Country (string), Age standardized (boolean). Format: CSV. Coverage: 198 countries, 1950-2011\n\nData Manipulation Code: ``` import pandas as pd import matplotlib.pyplot as plt # Load the data from the CSV file data = pd.read_csv('suicide_rates.csv') # Filter the data for the specific years of interest years_of_interest = [2000, 2005, 2010, 2015] data_filtered = data[data['Year'].isin

In [ ]:
compiled_metric.save("DSPy/compiled_metric-GPT")

In [26]:
compiled.save("DSPy/compiled_program-GPT")

[('text_to_data_manipulation.generate_data_manipulation', Predict(StringSignature(data_description, graph_description -> rationale, data_manipulation_code
    instructions='Generate data manipulation code from data and target graph descriptions'
    data_description = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Data Description:', 'desc': '${data_description}'})
    graph_description = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Graph Description:', 'desc': '${graph_description}'})
    rationale = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the data_manipulation_code}. We ...', '__dspy_field_type': 'output'})
    data_manipulation_code = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'output', 'prefix': 'Data Manipulation Code:', 'desc': '${data_manipulation_code}'})
))